# 프로젝트: 퍼스널 컬러 진단 - VGG16

## 데이터
1. 유명인 웜톤 100장, 쿨톤 100장  
2. 일반인 웜톤 490장, 쿨톤 186장  

## 목표: VGG16 아키텍처에 퍼스널컬러(웜/쿨) 적용하기  
참고 링크:https://eremo2002.tistory.com/55     

## (1) 라이브러리 및 데이터 불러오기

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import vgg16

import numpy as np
import cv2
import os
import glob
from PIL import Image

In [2]:
# 파일 불러오는 함수
def load_images_from_directory(directory_path):
    image_paths = glob.glob(directory_path + '/*.jpg')  # 디렉토리 내의 모든 jpg 파일 경로 찾기
    image_paths.extend(glob.glob(directory_path + '/*.jpeg'))  # 디렉토리 내의 모든 jpeg 파일 경로 추가
    image_paths = glob.glob(directory_path + '/*.JPG')  # 디렉토리 내의 모든 JPG 파일 경로 찾기

    images = []
    i = 0
    for image_path in image_paths:
        if i == 100:
            break
        image = Image.open(image_path)
        image_np = np.array(image)
        images.append(image_np)
        i += 1

    return images

In [3]:
# 모든 이미지 불러오기
directory_path = "/aiffel//aiffel/project/first-repository/data/four_seasons/train_warm_1"  # 디렉토리 경로
directory_path2 = "/aiffel/aiffel/project/first-repository/data/four_seasons/train_cool_1/cool_1"  # 디렉토리 경로
images = load_images_from_directory(directory_path)
images2 = load_images_from_directory(directory_path2)

length = len(images)
print(length)
print(len(images2))

# 이미지 리스트 합치기
images = np.concatenate((images, images2), axis=0)
print(len(images))

100
100
200


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


## (2) 데이터 전처리

In [4]:
def preprocess_image(image):
    # 이미지 크기 조절
    image = tf.image.resize(image, (224, 224))
    # 이미지를 [0, 1] 범위로 정규화
    image = image / 255.0
    return image

images = np.array([preprocess_image(image) for image in images])
print(images.shape)

(200, 224, 224, 3)


In [5]:
# labels에 대한 코드
num = len(images)
labels = np.zeros(num)

# 뒤쪽의 50개의 요소를 1로 변경
labels[length:] = 1

# 0이 웜톤 1이 쿨톤
print(len(images))
print(labels.shape)
print(labels)

200
(200,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]


## (3) train, val 분리하기

In [52]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(images,
                                                  labels,
                                                  test_size=0.3,
                                                  random_state=316)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

x_train:  (140, 224, 224, 3)
y_train:  (140,)
x_val:  (60, 224, 224, 3)
y_val:  (60,)


In [53]:
# 데이터를 텐서로 변환
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.int32)

## (4) 모델 정의 및 컴파일

In [54]:
from keras.models import Model
from keras import Input
from keras import optimizers, initializers, regularizers, metrics

dim1 = 32
dim2 = 64

input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')
 
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Flatten()(x)
x = layers.Dense(1024, kernel_initializer='he_normal')(x)
x = layers.Dense(512, kernel_initializer='he_normal')(x)
x = layers.Dense(64, kernel_initializer='he_normal')(x)
x = layers.Dense(32, kernel_initializer='he_normal')(x)
output_tensor = layers.Dense(2, activation='softmax')(x)
 
model = Model(input_tensor, output_tensor)
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 56, 56, 128)       0   

In [55]:
from keras.callbacks import ReduceLROnPlateau

# 콜백 생성
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.0000001)

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', #categorical_crossentropy , sparse_categorical_crossentropy
              metrics=['accuracy'])

## (5) 모델 학습하기

In [56]:
history = model.fit(x_train, y_train,
                    batch_size=16,
                    epochs=10,
                    validation_data=(x_val, y_val), 
                    callbacks=[reduce_lr])

Epoch 1/10
9/9 [==============================] - 4s 274ms/step - loss: 3948.4365 - accuracy: 0.4857 - val_loss: 76.0497 - val_accuracy: 0.5500
Epoch 2/10
9/9 [==============================] - 2s 240ms/step - loss: 73.2245 - accuracy: 0.5357 - val_loss: 69.6962 - val_accuracy: 0.4500
Epoch 3/10
9/9 [==============================] - 2s 239ms/step - loss: 67.4456 - accuracy: 0.6071 - val_loss: 64.7619 - val_accuracy: 0.5500
Epoch 4/10
9/9 [==============================] - 2s 243ms/step - loss: 63.0677 - accuracy: 0.4643 - val_loss: 61.0144 - val_accuracy: 0.4500
Epoch 5/10
9/9 [==============================] - 2s 243ms/step - loss: 59.6875 - accuracy: 0.5214 - val_loss: 58.0934 - val_accuracy: 0.4500
Epoch 6/10
9/9 [==============================] - 2s 237ms/step - loss: 57.0359 - accuracy: 0.5214 - val_loss: 55.7560 - val_accuracy: 0.4500
Epoch 7/10
9/9 [==============================] - 2s 233ms/step - loss: 54.8863 - accuracy: 0.5214 - val_loss: 53.8174 - val_accuracy: 0.4500
Epoc

## (6) 예측하기

In [57]:
pred = model.predict(x_val)
pred_class = np.argmax(pred, axis=1)
y_val = np.array(y_val)

In [58]:
print(pred_class)
print(y_val)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0
 0 0 1 0 1 0 0 1 0 0 1 1 0 0 0 1 1 0 1 1 1 0 1]


In [59]:
acc = np.mean(pred_class == y_val)
print('accuracy: %f' % (acc,))

accuracy: 0.450000


- 아키텍처를 간소화시키고 learning_rate를 조절하니 값이 다양하게 나오고 정확도가 0.3 -> 0.6까지 올랐다  
- 유명인 100장 시도  
1차 시도: 0.63(77)  
2차 시도: 40(99)  
3차 시도: 0.37(33)  
4차 시도: 0.53(316)  
- 데이터를 바꾸니까 0.6 -> 0.9까지 올랐다  
- 알반인 200장 시도  
1차 시도: 0.92(316)  
2차 시도: 0.65(99)  
3차 시도: 0.82(1004)  
4차 시도: 0.75(42)  
- 다시 VGG16의 본래 아키텍처로 다시 시도 (learning rate는 그대로 1e-6)  
1차 시도: 0.52(42)  
2차 시도: 0.35(55)  
3차 시도: 0.47(1004)  
4차 시도: 0.45(316)  
 -> 다시 기울기를 잃는 모습을 확인할 수 있다 그러므로 깊이를 축소시키는게 답이었다  

## (7) TEST

In [18]:
class_name = ["웜톤", "쿨톤"]

In [23]:
# test 불러오기 (낱개 버전)
test_path = "/aiffel/aiffel/project/first-repository/data/four_seasons/TEST/박은빈.jpg"
x_test = Image.open(test_path)
x_test = np.array(x_test)
print(x_test.shape)

# 배열의 크기 변환
x_test = x_test.reshape((1, x_test.shape[0], x_test.shape[1], 3))
print("변환된 배열 크기:", x_test.shape)

# 전처리(resize, 정규화)
x_test = preprocess_image(x_test)
print(x_test.shape)

# 만약 앱구현과 연결된다면 저장된 모델을 다시 돌릴 필요없이 저장된 최적의 모델을 사용하도록 수정해야함
# test
pred = model.predict(x_test)
pred_class = np.argmax(pred, axis=1)
print("당신은", class_name[int(pred_class)], "입니다~!")
print("웜톤일 확률: ", np.round(pred[:, 0], 3))
print("쿨톤일 확률: ", np.round(pred[:, 1], 3))

(275, 183, 3)
변환된 배열 크기: (1, 275, 183, 3)
(1, 224, 224, 3)
당신은 쿨톤 입니다~!
웜톤일 확률:  [0.488]
쿨톤일 확률:  [0.512]


In [159]:
# 박은빈 쿨톤, 강혜원 웜톤 맞춤!!